In [24]:
from src.model_fusion.model import UNET
from src.model_fusion.dataset import RSDataset
from src.model.dataset import PlanetDataset
from torch.utils.data import DataLoader
import torch

In [8]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
BATCH_SIZE = 16
NUM_WORKERS = 2
PIN_MEMORY = True
LOAD_MODEL = False
IMG_DIR = '../data/ai_data/val_images/'
MASK_DIR = '../data/ai_data/val_masks/'

In [10]:
ds = RSDataset(
    image_dir=IMG_DIR,
    mask_dir=MASK_DIR,
    transform=False)


loader = DataLoader(
    ds,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    pin_memory=PIN_MEMORY,
    shuffle=False)

In [26]:
rgb_images = PlanetDataset(
             image_dir=IMG_DIR,
             mask_dir=MASK_DIR, 
             transform=None)

In [28]:
model = UNET(in_channels=3, out_channels=1).to(DEVICE)


In [ ]:
with torch.no_grad():
    for x, y, z, m in loader:
        x = x.to(device)
        y = y.to(device)
        z = z.to(device)
        m = m.to(device).unsqueeze(1)
        preds = torch.sigmoid(model(x, y, z))
        preds = (preds > 0.5).float()